# <span style="color:darkblue"> Lecture 18 - Connecting to SQL via Python </span>

<font size = "5">

In the past lectures:

- We worked directly in SQL
- Imported data
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Import libraries

In [3]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Import Data

In [4]:
# Import the two car racing datasets

results = pd.read_csv("data_raw/results.csv")
races = pd.read_csv("data_raw/races.csv")
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Connect to SQL server

In [5]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'formula1',
    username = 'postgres',
    port = 5432,
    password  = '12345')

connection = create_engine(url_server)


<font size = "5">

Upload data to SQL

In [6]:
# Import the "results" table, with the name "results_sql"
# Con is an argument to specify the server connection
# "if_exists" is an option to replace the table if it already exists
# You can choose any name instead of "results_sql"

results.to_sql('results',
               con = connection,
               if_exists='replace',
               index=False)

# Import "races"
races.to_sql('races', 
             con = connection, 
             if_exists='replace',
             index=False)

102

<font size = "5">

Try it yourself!

- Upload the "circuits table" into SQL <br>
using ".to_sql()"

In [7]:
# Write your own code
circuits.to_sql('circuits',
               con = connection,
               if_exists='replace',
               index=False)

77

# <span style="color:darkblue"> II. "Escape" Characters </span>

<font size = "5">

- Backslash (\\) is known as an escape character
- Used for special operations on strings
- Great for working with text!

<font size = "5">

Multi-line strings (" \ ")

In [8]:
# Use a backslash to define strins over multiple lines
# Do not include a space after "\", otherwise it won't work.
example_string = "This is a string \
                  defined over multiple lines"
print(example_string)
example="this is a \
    defined"
print(example)

This is a string                   defined over multiple lines
this is a     defined


<font size = "5">

"Double quotes" inside string

In [9]:
# Use a backslash + quotation 

example_double = 'This will "supposedly" put double quotes inside a string'
print(example_double)

This will "supposedly" put double quotes inside a string


<font size = "5" >
'Single quotes' inside string

In [10]:
# There is no need for a backslash given single quotes 

example_single = "This will 'supposedly' put single quotes inside a string"
print(example_single)

This will 'supposedly' put single quotes inside a string


<font size = "5">

Try it yourself!

- Print a string ``` SELECT "driverId" FROM results; ``` <br> 
using backslash

In [11]:
# Write your own code


# <span style="color:darkblue"> III. Data operations </span>

<font size = "5">

- In SQL, operations are often called "queries"


<font size = "5">

Import Data from SQL

In [12]:
# Use "pd.read_sql()"
# - The first argument is string with instructions wrapped in text()
#- The second argument is the server connection

# Extract all data from a column
example1 = pd.read_sql(text("SELECT * FROM results;"), connection)

# Extract a subset of columns
example2 = pd.read_sql(text("SELECT points \
                             FROM results;"), connection)

# Subset based on a string condition
example3 = pd.read_sql(text("SELECT * \
                             FROM races \
                             WHERE name = 'Abu Dhabi Grand Prix';"), connection)

# Note: (i) Remember to include "\"  to be able to define
#           strings over multiple lines
#       (ii) We can include single quotations in the WHERE command
#            without any additional escape characters

<font size = "5">

Upper case columns

- In SQL syntax we use double quotes e.g. ``` "driverId" ```
- ```.read_sql()``` requires a string inside a string
- To do so, use escape characters, e.g.   ``` \"driverId\" ```

In [13]:
# Select a column
example4 = pd.read_sql(text("SELECT \"driverId\" \
                             FROM results;"), connection)

# Compute an aggregate statistic
example5 = pd.read_sql(text("SELECT AVG(points) as mean_points \
                             FROM results \
                             GROUP BY \"driverId\" ;"), connection)


<font size = "5">

Merge two datasets

In [14]:
# Merge 
# Reference the column \"driverId\" with escape characters
example6 = pd.read_sql(text("SELECT * \
                             FROM results \
                             LEFT JOIN races \
                             ON results.\"raceId\" = races.\"raceId\" ;"), connection)

# Merge a subset of columns
# Use "results_sql.*" to select all columns from the primary dataset
# Use "races_sql.date" to only select the "date" column from the secondary dataset 

example7 = pd.read_sql(text("SELECT results.*, races.date \
                             FROM results \
                             LEFT JOIN races \
                             ON results.\"raceId\" = races.\"raceId\" ;"), connection)


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- FROM results_sql compute the sum of points by <br>
"raceId" <br>

In [16]:
# Write your own code
example5 = pd.read_sql(text("SELECT SUM(points) as sum_points \
                             FROM results \
                             GROUP BY \"raceId\" ;"), connection)

hello1


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- Merge "races_sql" and the circuits table on "circuitId" <br>
tha you imported above using ```LEFT JOIN```

In [23]:
# Write your own
example7 = pd.read_sql(text("SELECT results.*, races.date \
                             FROM races \
                             LEFT JOIN circuits \
                             ON races.\"circuitId\" = circuits.\"circuitId\" ;"), connection)

ProgrammingError: (psycopg2.errors.UndefinedTable) 错误:  对于表"results",丢失FROM子句项
LINE 1: SELECT results.*, races.date                              FR...
               ^

[SQL: SELECT results.*, races.date                              FROM races                              LEFT JOIN circuits                              ON races."circuitId" = circuits."circuitId" ;]
(Background on this error at: https://sqlalche.me/e/20/f405)

# <span style="color:darkblue"> IV. Additional Material </span>

<font size = "5">

More about SQL syntax

https://www.w3schools.com/sql/